# Lopéz-Puertas et al. 2013

This notebook focusses on replicating the work of Lopéz-Puertas at al. (2013), where the researchers looked for PAHs making up the 3.3 μm emission recorded in VIMS data of Titan's upper atmosphere. This research used V1.20 of the NASA Ames PAH database for infrared spectroscopy and confirmed the presence of several PAHs

In [110]:
#import packages
import numpy as np
from amespahdbpythonsuite import observation
from amespahdbpythonsuite.amespahdb import AmesPAHdb
import matplotlib.pyplot as plt
from specutils import Spectrum1D


In [111]:
#read in the data file in ipac table format with the wavelength in microns and the flux in Jy
# data_file = "/Users/floorstikkelbroeck/Documents/Titan/titan_spec_900.ipac"
# data_file = "/Users/floorstikkelbroeck/Documents/Titan/titan_spec_950.ipac"
data_file = "/Users/floorstikkelbroeck/Documents/Titan/titan_spec_1000.ipac"
obs = observation.Observation(data_file)
obs.abscissaunitsto("1/cm") #convert the wavelength to wavenumbers

In [112]:
#load in the PAHdb with version 1.20 to mimic Lopéz-Puertas et al. 2013
# xml_file = "/Users/floorstikkelbroeck/Documents/Titan/PAHdb/pahdb-theoretical.xml"
xml_file = "/Users/floorstikkelbroeck/Documents/Titan/PAHdb/pahdb-theoretical-v1.20.xml"
pahdb = AmesPAHdb(
    filename=xml_file,
    check=False,
    cache=False,
)

                 AmesPAHdbPythonSuite
                 
                          by
                          
                Dr. Christiaan Boersma
                
                          and
                         
             Dr. Alexandros Maragkoudakis
             
               Dr. Matthew J. Shannanon
               
                  Dr. Joseph E. Roser
                 

          SUITE VERSION: 0.5.0.post62+g09a79ed         

        WEBSITE: HTTP://WWW.ASTROCHEM.ORG/PAHDB/       

          CONTACT: CHRISTIAAN.BOERSMA@NASA.GOV         

     PARSING DATABASE: THIS MAY TAKE A FEW MINUTES     

==
FILENAME                    : /var/folders/vv/2nq91_0d0lsfm6gcskyz__r40000gn/T/a80b922979ad89624d56b35e6a3a1f28.pkl
PARSE TIME                  : 0:00:02.784929
VERSION (DATE)              : 1.20 (2011-01-13)
COMMENT                     : 

This is the NASA Ames PAH IR Spectroscopic Database. The contents of
the database are described in Bauschlicher et al. (2010), Boersma 

The database version of 1.20 contains 604 PAHs, including neutrals, cations and anions. We will not include:
 - Charged particals (not anticipated at that height, and low efficiencies)
 - species containing Magnesium, iron, silicon, oxygen

This will reduce the dataset to 202 PAHs

In [113]:
# Retrieve the transitions from the database for the subset of PAHs.
uids = pahdb.search("neutral mg=0 fe=0 si=0 o=0 c<10")
transitions = pahdb.gettransitionsbyuid(uids)

In [114]:
#Check the number of PAHs in the database
show = transitions.get()
show.keys()
len(show['uids']) #should be 202 PAHs, I am missing one??

4

In [115]:
from SORCE import collect_irradiance_data
irradiance_data = collect_irradiance_data()
wavel = np.array(irradiance_data['august']['wavelengths'])
irrad = np.array(irradiance_data['august']['irradiances'])


In [116]:
#I have a spectrum for UV photons, how do i cascade this with the PAHdb
# Assuming you have a UV spectrum dictionary similar to the solar_spectrum dictionary
uv_spectrum = {
    "frequency": 1e7 / np.flip(wavel),  # frequency in nm
    "intensity": 1e-4 *np.flip(irrad * wavel**2) / (4 * np.pi)   # Replace with your UV spectrum intensities
}


In [117]:
#plot the converted data, frequency and intensity
# plt.plot(uv_spectrum['frequency'], uv_spectrum['intensity'])
# plt.xlabel('Frequency [cm^-1]')
# plt.ylabel('Intensity [W/m^2/Hz/sr]')
# plt.show()

In [118]:
# Cascade with the UV spectrum
transitions.cascade(uv_spectrum, star=True, stellar_model=True, convolved=True, multiprocessing=False, cache=False)

            APPLYING CASCADE EMISSION MODEL            

 STELLAR MODEL SELECTED: USING FIRST PARAMETER AS MODEL

          REBINNING STELLAR MODEL: 100 POINTS          

CALCULATED EFFECTIVE TEMPERATURE: 390.7633515171403 Kelvin

         CONVOLVING WITH ENTIRE RADIATION FIELD        

SPECIES                          : 1/4
UID                              : 493
MEAN ABSORBED ENERGY             : 3.371071970284385 +/- 0.7102877713772469 eV
MAXIMUM ATTAINED TEMPERATURE     : 1624.8640239938525 Kelvin
ENERGY CONSERVATION IN SPECTRUM  : 0.0
ELAPSED TIME                     : 0:00:06.341376
SPECIES                          : 2/4
UID                              : 495
MEAN ABSORBED ENERGY             : 3.371071970284385 +/- 0.7102877713772469 eV
MAXIMUM ATTAINED TEMPERATURE     : 1563.6202350978724 Kelvin
ENERGY CONSERVATION IN SPECTRUM  : 0.0
ELAPSED TIME                     : 0:00:06.589359
SPECIES                          : 3/4
UID                              : 485
MEAN ABSORBED ENERGY

/Users/floorstikkelbroeck/anaconda3/envs/titan/lib/python3.12/site-packages/amespahdbpythonsuite/transitions.py:942: RuntimeWarning: overflow encountered in power
  3458e-20 * 10.0 ** (-3.431 * wave)


In [ ]:
# Convolve the bands with a Gaussian with FWHM of 15 /cm.
transitions.shift(-15.0) 

spectrum = transitions.convolve(
    grid=obs.getgrid(), fwhm=15.0, gaussian=True, multiprocessing=False
)

In [ ]:
#check the obs array for inf or NaN values
nan_indices = np.where(np.isnan(obs.get()['spectrum'].flux.value))
print("Indices with NaN values:", nan_indices)
# np.all(np.isfinite(obs.get()))


In [ ]:
spectrum.get().keys()

In [ ]:
spectrum.get()['data']

In [ ]:
# Fit the spectrum
fit = spectrum.fit(obs)

In [ ]:
fit.plot(wavelength=True)
fit.plot(wavelength=True, residual=True)


In [ ]:
fit_info = fit.get()
fit_info.keys()
fit_info['weights']

Get the molecular structure of the UID of the most present molecules in the fit

In [ ]:
# Get the weights from fit_info
weights = fit_info['weights']

# Sort the uids by their weights in descending order
sorted_uids_by_weight = sorted(fit_info['uids'], key=lambda uid: weights[uid], reverse=True)

# Print the uids and their formulas
for uid in sorted_uids_by_weight:
    species_uid = pahdb.getspeciesbyuid([uid]).get()
    mol_for = species_uid['data'][uid]['formula']
    print(f'The uid is: {uid}, the formula is: {mol_for}, the weight is: {weights[uid]}')


In [16]:
#Shows contibution of large vs small particles
# fit.plot(wavelength=True, size=True)

In [17]:
#shows contribution of neutrals or charged molecules (not applicable now because only neutrals considered)
# fit.plot(wavelength=True, charge=True)

In [ ]:
#contribution from pure or nitrogen containing PAHs
fit.plot(wavelength=True, composition=True)

In [ ]:
# Predict 3.15 - 3.40 µm spectrum
transitions.intersect(fit.getuids())
spectrum = transitions.convolve(gaussian=True, multiprocessing=False, grid=obs.getgrid())
coadded = spectrum.coadd(weights=fit.getweights())
coadded.plot()

In [19]:
#Get the fit spectrum so i can plot it in the next cell
spectrum_output = fit.observation
flux_values = spectrum_output.flux.value

In [22]:
#get the original observed spectrum
obs_output = obs.get()
original_obs = obs_output['spectrum'].flux.value
original_obs_x = obs_output['spectrum'].spectral_axis.value
# original_obs_x = 1e4/original_obs_x
# help(obs)

In [ ]:
plt.plot(coadded.grid, coadded.data[0], label='Coadded Spectrum', color = 'purple')
# Plot the fit spectrum
plt.plot(fit.grid, flux_values, label='Fit Spectrum', color = 'orange')
plt.plot(original_obs_x, original_obs, label='Original Spectrum', color = 'blue')
#invert the x-axis to show the wavenumbers in increasing order
plt.gca().invert_xaxis()

# Add labels and legend
plt.xlabel('Wavelength (µm)')
plt.ylabel('Intensity')
plt.legend()
plt.title('Coadded and Fit Spectra')

# Show the plot
plt.show()

# Monte Carlo Fit

In [ ]:
# Fit the spectrum using Monte Carlo approach.
mcfit = spectrum.mcfit(obs, samples=1024, multiprocessing=False)

In [ ]:
# mcfit.plot(wavelength=True)
mcfit.plot(wavelength=True, residual=True)
mcfit.plot(wavelength=True, size=True)
mcfit.plot(wavelength=True, charge=True)
# mcfit.plot(wavelength=True, composition=True)
# mcfit.plot(wavelength=True, save=True, ftype="pdf")

In [ ]:
mcfit_info = mcfit.get()
mcfit_info

In [ ]:
mcfit.getfit()

In [ ]:
mcfit.getbreakdown()